# 用GPU训练 笔记
教程视频链接：https://www.bilibili.com/video/BV1hE411t7RN

这篇笔记对应视频合集中的
- 利用GPU训练（一）
- 利用GPU训练（二）

## 方法1
调用以下元素的cuda方法：
1. 网络模型
2. 数据（输入，标注）
3. 损失函数

## 方法2（更常用）
定义设备`device = torch.device("cuda")`

若有多个cuda设备``device = torch.device("cuda:0")`

对上述元素调用`to(device)`方法

In [10]:
import torchvision

train_data = torchvision.datasets.CIFAR10(root='../dataset', train=True, transform=torchvision.transforms.ToTensor(), download=False)
test_data = torchvision.datasets.CIFAR10(root='../dataset', train=False, transform=torchvision.transforms.ToTensor(), download=False)

#数据集长度
train_data_size = len(train_data)#50000
test_data_size = len(test_data)#10000
print(train_data_size, test_data_size)

import torch

train_dataloader = torch.utils.data.DataLoader(dataset=train_data, batch_size=64)
test_dataloader = torch.utils.data.DataLoader(dataset=test_data, batch_size=64)

#此处类定义代码可以放在另一个文件中
class Tudui(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2), #这里一定记得加逗号
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=1024, out_features=10)
        )
    def forward(self, x):
        return self.model(x)
tudui = Tudui()

#------------------------------------------
#方法1
tudui = tudui.cuda()
#------------------------------------------

#损失函数
loss_fn = torch.nn.CrossEntropyLoss()

#------------------------------------------
#方法2
device = torch.device("cuda")
loss_fn = loss_fn.to(device)
#------------------------------------------

#优化器
learning_rate = 0.001#1e-2
optimizer = torch.optim.SGD(params=tudui.parameters(), lr=learning_rate)

50000 10000


In [ ]:
total_train_step = 0
total_test_step = 0
epoch = 100

for i in range(epoch):
    epoch_loss = 0
    for data in train_dataloader:
        inputs, targets = data
        #------------------------------------------
        inputs = inputs.cuda()
        targets = targets.cuda()
        #------------------------------------------
        outputs = tudui(inputs)
        optimizer.zero_grad()
        result_loss = loss_fn(outputs, targets)
        result_loss.backward()
        optimizer.step()
        epoch_loss += result_loss
        total_train_step += 1
    print("第{0}轮，第{1}步，loss为{2}".format(i, total_train_step, epoch_loss))
    
    #测试
    total_test_loss = 0
    with torch.no_grad():
        for data in test_dataloader:
            img, targets = data
            #------------------------------------------
            if torch.cuda.is_available():#良好的编程习惯
                img = img.cuda()
                targets = targets.cuda()
            #------------------------------------------
            outputs = tudui(img)
            total_test_loss += loss_fn(outputs, targets)
    print("测试集上的loss为{0}".format(total_test_loss))
    total_test_step += 1